In [1]:
%matplotlib inline
#Import  necessary tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


# Reformat graphics
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (30, 14)

<font size = 5> Import and prepare GTD Data </font>

In [2]:
#Read in the data set
gtd = pd.read_csv('/Users/johntate/Desktop/DTSC 3601/Project 2/terrorism_predictive_analytics/Data Sets/Raw Data Sets/gtdDataSet.csv')

/Users/johntate/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4,31,33,54,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Select for relevant columns
gtd = gtd[['iyear', 'imonth', 'summary', 'country', 'country_txt',  'gname', 'success', 'targtype1', 'nperps', 'nkill', 'nwound','weaptype1', 'weaptype1_txt',  'weapsubtype1']]

In [4]:
gtd.shape

(201183, 14)

In [5]:
#Create a list of relevant countries to filter for Canada, Germany, France, UK, Italy, Japan, USA
countries_list = [38, 75, 69, 603, 98, 101, 217]
boolean_series = gtd.country.isin(countries_list)

In [6]:
#Select for attacks that occurred in relevant countries
gtd = gtd[boolean_series]

In [7]:
#Check to make sure that only listed countries appear
gtd.country.value_counts()

603    5424
217    3004
69     2726
98     1589
75      775
101     405
38      112
Name: country, dtype: int64

In [8]:
#Filter to relevant years
gtd = gtd[(gtd['iyear'] >= 2006) & (gtd['iyear'] < 2020)]

In [9]:
#Check shape
gtd.shape

(2193, 14)

In [10]:
#Drop any duplicate rows, since we subsetted columns there should be no duplicates
gtd.drop_duplicates(inplace=True)
gtd.shape

(2125, 14)

In [11]:
#Check the distribution of attack types
gtd.weaptype1.value_counts()

8     793
6     766
5     322
9     155
10     31
13     18
12     13
2      12
11     11
1       4
Name: weaptype1, dtype: int64

<font size = 5> Begin working with OECD data and joining to GTD </font>

In [12]:
#Read in OECD Leading Indicators data set
oecd_df = pd.read_csv('OECD_Leading_DataSet.csv')

In [13]:
#Create a column in OECD that provides the different between every observation from three months prior as a %
oecd_df['delta'] = oecd_df.groupby(['LOCATION'])['Value'].diff(3)

# reset the index to return the dataframe to its original form
oecd_df.reset_index(drop=True, inplace=True)

# show the updated dataframe
oecd_df.head(10)

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes,delta
0,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-02,100.2482,NaN,NaN
1,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-03,100.2449,NaN,NaN
2,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-04,100.2153,NaN,NaN
3,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-05,100.1706,NaN,-0.0776
4,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-06,100.1293,NaN,-0.1156
5,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-07,100.0971,NaN,-0.1182
6,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-08,100.0654,NaN,-0.1052
7,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-09,100.0354,NaN,-0.0939
8,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-10,100.0230,NaN,-0.0741
9,GBR,CLI,AMPLITUD,LTRENDIDX,M,2005-11,100.0548,NaN,-0.0106


In [14]:
# show the first 5 rows for the first 3 countries to show that a delta was calculated
oecd_df[oecd_df.LOCATION.isin(oecd_df.LOCATION.unique()[:3])].set_index(['LOCATION', 'TIME'])[['Value', 'delta']].groupby(level=0).head(5)

Value    delta
LOCATION TIME                       
GBR      2005-02  100.24820      NaN
         2005-03  100.24490      NaN
         2005-04  100.21530      NaN
         2005-05  100.17060 -0.07760
         2005-06  100.12930 -0.11560
ITA      2005-02   99.84733      NaN
         2005-03   99.65719      NaN
         2005-04   99.44277      NaN
         2005-05   99.25113 -0.59620
         2005-06   99.13202 -0.52517
JPN      2005-02  100.59520      NaN
         2005-03  100.56010      NaN
         2005-04  100.54800      NaN
         2005-05  100.56660 -0.02860
         2005-06  100.62440  0.06430

In [15]:
#Create a year-month column in terror data frame so that we can join OECD data
gtd['year-month'] = gtd.apply(lambda x: f"{x.iyear}-{str(x.imonth).zfill(2)}", axis=1)

In [16]:
# manually create a dataframe holding country code conversions
country_codes = pd.DataFrame([
    ['United Kingdom', 'GBR'],
    ['Italy', 'ITA'],
    ['Japan', 'JPN'],
    ['France', 'FRA'],
    ['United States', 'USA'],
    ['Germany','DEU'],
    ['Canada', 'CAN']
], columns=['country_name', 'country_code'])
country_codes

,country_name,country_code
0,United Kingdom,GBR
1,Italy,ITA
2,Japan,JPN
3,France,FRA
4,United States,USA
5,Germany,DEU
6,Canada,CAN


In [17]:
# add a column for country codes in order to merge the original dataframe
gtd = gtd.merge(country_codes, how='left', left_on='country_txt', right_on='country_name').drop(columns='country_name')
gtd.head(10)

,iyear,imonth,summary,country,country_txt,gname,success,targtype1,nperps,nkill,nwound,weaptype1,weaptype1_txt,weapsubtype1,year-month,country_code
0,2006,1,01/06/2006: A bomb exploded near the army recr...,69,France,Unknown,1,4,NaN,0.0,0.0,6,Explosives,16.0,2006-01,FRA
1,2006,1,01/17/2006: A $3 million dollar home under con...,217,United States,Earth Liberation Front (ELF),1,14,NaN,0.0,0.0,8,Incendiary,18.0,2006-01,USA
2,2006,1,01/17/2006: A butcher shop belonging to a Musl...,69,France,Unknown,1,1,NaN,0.0,0.0,6,Explosives,16.0,2006-01,FRA
3,2006,1,01/19/2006: The flat of a retired gendarme now...,69,France,Corsican National Liberation Front (FLNC),1,3,NaN,0.0,0.0,6,Explosives,16.0,2006-01,FRA
4,2006,1,01/19/2006: A villa under construction in Sago...,69,France,Corsican National Liberation Front (FLNC),0,14,NaN,0.0,0.0,6,Explosives,14.0,2006-01,FRA
5,2006,1,"01/20/2006: In Sagone, Corsica, a home under c...",69,France,Unknown,1,14,NaN,0.0,0.0,6,Explosives,16.0,2006-01,FRA
6,2006,1,"01/22/2006: A tax office in Aix-en-Provence, F...",69,France,Corsican National Liberation Front (FLNC),1,2,NaN,1.0,0.0,6,Explosives,16.0,2006-01,FRA
7,2006,1,"01/23/2006: In a series of related incidents, ...",69,France,Unknown,1,14,NaN,0.0,0.0,6,Explosives,16.0,2006-01,FRA
8,2006,1,"01/23/2006: In a series of related incidents, ...",69,France,Unknown,1,1,NaN,0.0,0.0,6,Explosives,16.0,2006-01,FRA
9,2006,1,"01/24/2006: In a series of related incidents, ...",69,France,Unknown,1,14,NaN,0.0,0.0,6,Explosives,16.0,2006-01,FRA


In [18]:
# Check to make sure that all countries have a country code
gtd[gtd['country_code'].isna()].country_txt.unique()

array([], dtype=object)

In [19]:
gtd.columns


Index(['iyear', 'imonth', 'summary', 'country', 'country_txt', 'gname',
       'success', 'targtype1', 'nperps', 'nkill', 'nwound', 'weaptype1',
       'weaptype1_txt', 'weapsubtype1', 'year-month', 'country_code'],
      dtype='object')

In [20]:
oecd_df.columns

Index(['LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'TIME',
       'Value', 'Flag Codes', 'delta'],
      dtype='object')

In [21]:
#Remove oced columns that we won't use before joining
oecd_df.drop(['MEASURE', 'INDICATOR', 'FREQUENCY', 'Flag Codes', 'SUBJECT'], axis = 1, inplace = True )

In [22]:
# merge the two dataframes 
merged_gtd = gtd.merge(oecd_df, left_on=['country_code', 'year-month'], right_on=['LOCATION', 'TIME']).drop(['LOCATION', 'TIME'], axis = 1)

merged_gtd.columns

Index(['iyear', 'imonth', 'summary', 'country', 'country_txt', 'gname',
       'success', 'targtype1', 'nperps', 'nkill', 'nwound', 'weaptype1',
       'weaptype1_txt', 'weapsubtype1', 'year-month', 'country_code', 'Value',
       'delta'],
      dtype='object')

In [24]:
#Look at a random sample of data to make sure that everything looks correct
merged_gtd.sample(10)

,iyear,imonth,summary,country,country_txt,gname,success,targtype1,nperps,nkill,nwound,weaptype1,weaptype1_txt,weapsubtype1,year-month,country_code,Value,delta
412,2011,12,12/12/2011: A parcel bomb addressed to the Gre...,69,France,Informal Anarchist Federation,0,7,-99.0,0.0,0.0,6,Explosives,9.0,2011-12,FRA,100.02190,-0.75670
63,2007,2,01/18/2007: An explosives-laden letter detonat...,603,United Kingdom,Anti-Government extremists,1,1,1.0,0.0,2.0,6,Explosives,9.0,2007-02,GBR,102.82860,0.62660
1755,2018,4,04/10/2018: An explosive device detonated at a...,98,Italy,Anarchists,1,1,-99.0,0.0,0.0,6,Explosives,14.0,2018-04,ITA,101.67360,-0.39530
480,2012,8,08/05/2012: An assailant opened fire on a Sikh...,217,United States,White supremacists/nationalists,1,15,1.0,7.0,4.0,5,Firearms,5.0,2012-08,USA,99.56949,-0.23948
440,2012,4,04/16/2012: Two explosive devices were thrown ...,603,United Kingdom,Loyalists,1,14,-99.0,0.0,0.0,8,Incendiary,19.0,2012-04,GBR,97.88142,0.04198
231,2009,9,"09/04/2009: On Friday night around 0330, an as...",217,United States,Earth Liberation Front (ELF),1,16,-99.0,0.0,0.0,11,Sabotage Equipment,NaN,2009-09,USA,97.32363,1.72233
1787,2018,7,07/04/2018: An assailant punched a man with a ...,217,United States,Unknown,1,14,-99.0,0.0,1.0,9,Melee,22.0,2018-07,USA,100.68680,-0.11680
764,2014,5,05/01/2014: A pipe bomb was discovered and rem...,603,United Kingdom,Unknown,0,20,-99.0,0.0,0.0,6,Explosives,31.0,2014-05,GBR,102.52980,0.07500
1812,2018,7,07/11/2018: Assailants attacked firefighters i...,603,United Kingdom,Unknown,1,13,-99.0,0.0,0.0,13,Unknown,NaN,2018-07,GBR,99.33531,-0.60926
1942,2019,1,01/29/2019: Assailants set fire to a TDF trans...,69,France,Anarchists,1,16,-99.0,0.0,0.0,8,Incendiary,20.0,2019-01,FRA,99.16489,-0.41672


<font size = 5> Begin working with OECD immigration data and join to GTD </font>

In [70]:
#Import OECD immigration data
oecd_immigration_df = pd.read_csv('OECD_Migration.csv')

In [71]:
oecd_immigration_df.head(5)

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,STOCKFPOP,TOT,NBR,A,2006,5031630,NaN
1,AUS,STOCKFPOP,TOT,NBR,A,2007,5233250,NaN
2,AUS,STOCKFPOP,TOT,NBR,A,2008,5477900,NaN
3,AUS,STOCKFPOP,TOT,NBR,A,2009,5729880,NaN
4,AUS,STOCKFPOP,TOT,NBR,A,2010,5881360,NaN


In [72]:
#Import OECD population data
oecd_migration_df = pd.read_csv('OECD_Migration.csv')

In [73]:
#Create boolean series indicating whether each country is to be selected or not
country_boolean = oecd_migration_df['LOCATION'].isin(country_codes.country_code)

In [76]:
#Subset migration data frame for desired countries and display value counts to check distribution
oecd_migration_df = oecd_migration_df[country_boolean]
oecd_migration_df.LOCATION.value_counts()

<ipython-input-76-f328b9ba3303>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  oecd_migration_df = oecd_migration_df[country_boolean]


DEU    15
GBR    15
FRA    15
USA    14
ITA    12
CAN     3
Name: LOCATION, dtype: int64

<font size = 5> ***Ask Dr. Najjar about how to handle missing immigration level observations*** </font>

In [79]:
#reset index to return df to it's original form
oecd_migration_df.reset_index()

,index,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,45,CAN,STOCKFPOP,TOT,NBR,A,2006,6186950,NaN
1,46,CAN,STOCKFPOP,TOT,NBR,A,2011,6775765,NaN
2,47,CAN,STOCKFPOP,TOT,NBR,A,2016,7540830,NaN
3,79,FRA,STOCKFPOP,TOT,NBR,A,2006,6910060,NaN
4,80,FRA,STOCKFPOP,TOT,NBR,A,2007,7017207,NaN
...,...,...,...,...,...,...,...,...,...
69,300,USA,STOCKFPOP,TOT,NBR,A,2015,42390705,NaN
70,301,USA,STOCKFPOP,TOT,NBR,A,2016,43289646,NaN
71,302,USA,STOCKFPOP,TOT,NBR,A,2017,43738901,NaN
72,303,USA,STOCKFPOP,TOT,NBR,A,2018,44525458,NaN


In [81]:
#Read in the OECD population CSV
oecd_population_df = pd.read_csv('OECD_population.csv')

In [83]:
#Inspect the format of the data frame to prepare for joining
oecd_population_df.head()

,LOCATION,Country,SEX,Sex,AGE,Age,TIME,Time,Value,Flag Codes,Flags
0,AUS,Australia,W,Women,TOTAL,Total,2006,2006,10291542.0,NaN,NaN
1,AUS,Australia,W,Women,TOTAL,Total,2007,2007,10473986.0,NaN,NaN
2,AUS,Australia,W,Women,TOTAL,Total,2008,2008,10677154.0,NaN,NaN
3,AUS,Australia,W,Women,TOTAL,Total,2009,2009,10890856.0,NaN,NaN
4,AUS,Australia,W,Women,TOTAL,Total,2010,2010,11063919.0,NaN,NaN


In [ ]:
oecd_population_df

In [87]:
#Perform a left join to merge the population data frame with the migration data frame (migration is left)
oecd_migration_df = oecd_migration_df.merge(oecd_population_df, left_on = ['LOCATION'], right_on = ['LOCATION']).drop(['INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY, TIME_x', 'Flag Codes_x', ])

In [88]:
oecd_migration_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME_x,Value_x,Flag Codes_x,Country,SEX,Sex,AGE,Age,TIME_y,Time,Value_y,Flag Codes_y,Flags
0,CAN,STOCKFPOP,TOT,NBR,A,2006,6186950,NaN,Canada,W,Women,TOTAL,Total,2006,2006,16426415.0,NaN,NaN
1,CAN,STOCKFPOP,TOT,NBR,A,2006,6186950,NaN,Canada,W,Women,TOTAL,Total,2007,2007,16590173.0,NaN,NaN
2,CAN,STOCKFPOP,TOT,NBR,A,2006,6186950,NaN,Canada,W,Women,TOTAL,Total,2008,2008,16772940.0,NaN,NaN
3,CAN,STOCKFPOP,TOT,NBR,A,2006,6186950,NaN,Canada,W,Women,TOTAL,Total,2009,2009,16965482.0,NaN,NaN
4,CAN,STOCKFPOP,TOT,NBR,A,2006,6186950,NaN,Canada,W,Women,TOTAL,Total,2010,2010,17157066.0,NaN,NaN


To Do:
- Find immigration data and tie it in
- Create DF for election proximity and create a categorical variable for election proximity in terror DF
- Merge ACLED df
- Create list of visualizations needed and generate them
- Determine whether any variables need to be normalized prior to running analysis
- Figure out whether text analysis is useful
- Run statistical models with CV